In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.chrome.service import Service
from chromedriver_py import binary_path # this will get you the path variable

import time
import pandas as pd
import random 
import csv 
import math
from pathlib import Path


In [2]:
option = webdriver.ChromeOptions()
option.add_argument('log-level=3')
option.add_argument('--ignore-certificate-errors-spki-list')
option.add_argument('--ignore-ssl-errors')
# options.add_argument('--headless')
option.add_argument('--disable-gpu')
#service_object = Service(binary_path)
driver = webdriver.Chrome(service=Service(), options=option) 

In [3]:
#option = webdriver.FirefoxOptions()
#option.set_preference("browser.download.manager.showWhenStarting", False)
#option.set_preference('browser.helperApps.neverAsk.saveToDisk',
                      # 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                       #';application/vnd.ms-excel'
                      # ';application/xls'
                      # ';application/xlsx')
#option.set_preference("permissions.default.stylesheet", 2)
#option.set_preference("permissions.default.image", 2)
#option.set_preference("javascript.enabled", True)


In [4]:
driver= webdriver.Chrome(options=option)
driver.maximize_window()
from selenium.common.exceptions import TimeoutException

t = time.time()
driver.set_page_load_timeout(500)

try:
    driver.get('https://dmf.gujarat.gov.in/Home/Collection')
except TimeoutException:
    driver.execute_script("window.stop();")
#print('Time consuming:', time.time() - t)

assert "DMF" in driver.title
time.sleep(1)




In [5]:
main_df = pd.DataFrame(index=[], columns =['Sr', 'District', 'Collection_Amount', 'Category', 'Month', 'Year']) 
master_df1 = pd.DataFrame(index=[], columns =['Sr', 'District',  'Collection_Amount', 'Category', 'Month', 'Year']) 
master_df2 = pd.DataFrame(index=[], columns =['Sr', 'District',  'Collection_Amount', 'Category', 'Month', 'Year']) 


In [6]:
Financial_year_path = driver.find_elements(By.XPATH, '//*[@id="mineralCollection_Year"]/option[position()>1]')
Financial_year = [t.get_attribute("textContent") for t in Financial_year_path]
print(Financial_year)
len_year = len(Financial_year)+1

for year in range(1, len_year, 1): 
    Fin_year_select = Select(WebDriverWait(driver, 10).until(ec.element_to_be_clickable((By.ID,"mineralCollection_Year" ))))
    Fin_year_select.select_by_index(year)

    month_path = driver.find_elements(By.XPATH, '//*[@id="mineralCollection_Month"]/option[position()>1]')

    Financial_month = [te.get_attribute("textContent") for te in month_path]
    len_month = len(Financial_month)+1
    for month in range(1, len_month, 1):
        month_select = Select(WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.ID, "mineralCollection_Month"))))

        month_select.select_by_index(month)

        #click on view 
        driver.find_element(By.XPATH, '//*[@id="btnsubmit"]').click()

        #click on major view table 


        a= ActionChains(driver)
        element1 =  driver.find_element(By.XPATH, '//*[@id="radio1"]')
        a.move_to_element(element1)
        a.click(on_element=element1).perform()
        time.sleep(5)

        text_warning = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div/div[2]/div/table/tbody/tr[2]/td').get_attribute("textContent")

        print(text_warning)

        text_var = "There is no item to display."
        if text_warning ==text_var: 

           temp_df1 = pd.DataFrame(index=[], columns=[])
        else:
           tables = WebDriverWait(driver, 20).until(ec.presence_of_all_elements_located((By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div/div[2]/div/table')))

           for table in tables:
               df_st1= pd.read_html(table.get_attribute('outerHTML'))

           temp_df1= df_st1[0]
           len_temp = len(temp_df1.index)
           temp_cat = ["Major"]*len_temp 
           temp_year = [Financial_year[year-1]]*len_temp 
           temp_month = [Financial_month[month-1]]*len_temp
           temp_df1["Category"] = temp_cat
           temp_df1['Year'] =  temp_year
           temp_df1['Month']= temp_month
           master_df1 = pd.concat([master_df1, temp_df1])
      
        a.reset_actions()
        element2 = driver.find_element(By.XPATH, '//*[@id="radio2"]')
        a.move_to_element(element2)
        a.click(on_element=element2).perform()
        time.sleep(5)
        text_warning = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div/div[2]/div/table/tbody/tr[2]/td').get_attribute("textContent")
        print(text_warning)
        text_var = "There is no item to display."

        if text_warning ==text_var: 

           temp_df2 = pd.DataFrame(index=[], columns=[])
        else:
           tables = WebDriverWait(driver, 20).until(ec.presence_of_all_elements_located((By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div/div[2]/div/table')))

           for table in tables:
               df_st2= pd.read_html(table.get_attribute('outerHTML'))

           temp_df2= df_st2[0]
           len_temp = len(temp_df2.index)
           temp_cat = ["Minor"]*len_temp 
           temp_year = [Financial_year[year-1]]*len_temp 
           temp_month = [Financial_month[month-1]]*len_temp
           temp_df2["Category"] = temp_cat
           temp_df2['Year'] =  temp_year
           temp_df2['Month']= temp_month
           master_df2 = pd.concat([master_df2, temp_df2])

        main_df = pd.concat([master_df1, master_df2])


           


    



['2016 - 2017', '2017 - 2018', '2018 - 2019', '2019 - 2020', '2020 - 2021', '2021 - 2022', '2022 - 2023', '2023 - 2024']
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no item to display.
There is no i

In [7]:
path = "C:/Users/niranjan.kumar/Desktop/DMFT_Scraper/"
main_df.to_csv(path+'DMFT_Collection_Gujarat.csv', index=False)